In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import concurrent.futures
import pickle

### Generate transverse momenta

This example shows the simplest path to use mg5qs functions; in subsequent examples each step will be discussed in more detail. 

This example includes:
- loading a MadGraph model
- single-thread processing
- parallel processing
- storing results
- consolidating results from mulitiple runs
- plotting results using standard libraries
- computing statistics using standard libraries

### 1. Load a MadGraph model

**1.1 load environment variables; use MadGraph to create output directories and default cards**

The enviroment variables are loaded in mg5qs_import, but it is done here explicitly in case they have not been setup yet.

In [ ]:
# these environment variables are required, see mg5qs/README
MG5_PATH = Path(os.getenv('MG5QS_MG5_PATH'))      # where is madgraph?
INPUT_PATH = Path(os.getenv('MG5QS_INPUT_PATH'))  # where is model's proc_card? 
MG5_PATH, INPUT_PATH

In [ ]:
# now, import mg5qs
import mg5qs_import as qs

In [ ]:
# create madgraph output directory structure
output_name, FRAMEWORK_PATH = qs.run_MG5(qs.MG5_PATH, qs.INPUT_PATH, proc_card_name='proc_card.dat')
output_name, FRAMEWORK_PATH

**1.2 at this point, cards could be edited (manually) using qs.edit_card or (programatically) using the ParamCard API:**

In [ ]:
# example: the ParamCard API can abstract cards and set values for paramaters
card = qs.ParamCard(FRAMEWORK_PATH)
# ...set_value goes here...
card

### 2. Generate LHE files

Run MadGraph to generate Les Houches Event (LHE) files; these files will then by processed into hadron showers using Pythia.

In [ ]:
RUNS = 4

for n in range(RUNS):
    qs.generate_LHE(card, FRAMEWORK_PATH)

In [ ]:
LHEs = qs.get_LHEs(FRAMEWORK_PATH)
LHEs

### 3. Generate transverse momenta

**3.1 single-threaded operation**

Calling qs.generate_pT for a specified particle ID will:
- run Pythia
- intercept events that include the specified particle
- calculate transverse momenta
- return status and numpy array of discrete momenta values

In [ ]:
TAU = 15  # see MadGraph docs for more particle ID numbers 
result, pTs = qs.generate_pT(TAU, LHEs[0])  # call Pythia, capture pTs for TAU

In [ ]:
print(result) # shows total number of Tau particles and process return status code
if result['status'] != 0:
    print('something went wrong')

In [ ]:
# results from small single-threaded sample run
plt.hist(pTs, bins=30, log=True)
plt.title('Distribution of Transverse Momenta of Tau Particles')
plt.show()

**3.2 Multi-threaded operation**

Use Python's concurrent package to run Pythia processes in parallel, storing partial results in pickle files:

In [ ]:
import concurrent.futures
cpu_cores = 10  # set an appropriate value based on CPU hardware
results = np.array([])

def process_LHE(LHE, PID=15):
    return qs.generate_pT(PID, LHE)

with concurrent.futures.ProcessPoolExecutor(max_workers=cpu_cores) as executor:
    futures = {executor.submit(process_LHE, LHE): LHE for LHE in LHEs}
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        results = np.concatenate((results, result[1]))  # result[1] = pT data
        # ProcessPoolExecutor takes care of thread saftey, more on this in example_3

In [ ]:
# draw result of combined runs 
# notice this run is less noisy than last time because it contains data from 4 times more events 
plt.hist(results, bins=30, log=True)
plt.title('Distribution of Transverse Momenta of Tau Particles')
plt.xlabel("pT (GeV)")
plt.show()

**3.3 use standard library to calculate statistics**

Now that the data is Python-exposed any library can be applied to any end. Here is a simple comparison test from **scipy**:

In [ ]:
n = pTs.shape[0] // 2            # divide observations in half
stats.ks_2samp(pTs[0:n],pTs[n:]) # do these two samples come from the same distribution?